In [ ]:
pip install scikit-learn
pip install joblib

SyntaxError: ignored

In [ ]:
import os 
os.chdir('C:\\Users\\q42\\Thesis\\data')
os.getcwd()

'C:\\Users\\q42\\Thesis\\data'

In [ ]:
import numpy as np
# from sklearn.model_selection import cross_val_score # (not needed now)

In [ ]:
# windowsOSFileDatgenerator

# def save_obj(obj):
#     with open('data_preprocessed_python/all_32.dat', 'wb') as f:
#         pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# import numpy as np

# class values(np.ndarray):
#     def __new__(cls, *args, **kwargs):
#         return np.array(*args, **kwargs).view(values)
#     def index(self, value):
#         return np.where(self == value)
# only used in riemann classifier for covariance matrix building 

In [ ]:
dat_original_location = 'C:\\Users\\q42\\Thesis\\data\\s'
# files are named s1, s2.., s32
dat_new_location = 'C:\\Users\\q42\\q42\\Thesis\\data\\all_32.dat'
# create a single file with everything merged

data = {}
import pickle
def load_obj(name):
    with open(name + '.dat', 'rb') as f:
        return pickle.load(f, encoding='latin1')

for a in range(1, 33):
    if a < 10:
        participantId = "0" + str(a)ta
    else:
        participantId = str(a)

    dummy = dat_original_location + participantId
    exp_data = load_obj(dummy)

    # each parse list the number
    print("Subject: ", a)
    data[a] = exp_data


In [ ]:
def save_obj(obj, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)
def load_obj(name):
    with open(name, 'rb') as f:
        return pickle.load(f, encoding='latin1')
save_obj(data, 'C:\\Users\\q42\\Thesis\\data\\all_32.dat')
print(load_obj(dat_new_location))

# now eeg data for all 32 participants is in a single dictionary
# starting with labels and the values for arousal, valence, dominance, liking

In [ ]:
# here ends first function
##########################
##########################
##########################
##########################
##########################
##########################
# first end ok

In [ ]:
os.chdir('C:\\Users\\q42\\Thesis\\')

In [ ]:
dat_new_location = 'C:\\Users\\q42\\Thesis\\data\\all_32.dat'
dummy = load_obj(dat_new_location)
subjects = range(1, 33)
videos = range(0, 40)
channels = range(1, 33)
# Our data for the ml
X = []  # signal-feature array
y = []  # emotional features

In [ ]:
import numpy as np
import scipy.signal


In [ ]:
# Copyright (c) 2018 paulmorio
# https://github.com/paulmorio/deap_analysis
def eeg_bands(signal):
    # returns the alpha, theta, beta
    eeg = signal
    pi = np.pi
    y = np.array(eeg)  # faster array
    L = len(eeg)       # signal length
    fs = 128.0     # frequency sampling 128Hz
    T = 1/fs           # sample time
    t = np.linspace(1,L,L)*T

    f = fs*np.linspace(0,L/10,L/10)/L  # single side frequency vector, real frequency up to fs/2
    Y = np.fft.fft(y)

    filtered = []
    b= [] # store filter coefficient
    cutoff = [0.5,4.0,8.0,12.0,30.0] # delta, alpha, theta, beta bands

    for band in range(0, len(cutoff)-1):
        wl = 2*cutoff[band]/fs*pi
        wh = 2*cutoff[band+1]/fs*pi
        M = 512      # Set number of weights as 128
        bn = np.zeros(M)

        for i in range(0, M):     # Generate bandpass weighting function
            n = i-M/2       # Make symmetrical
            if n == 0:
                bn[i] = wh/pi - wl/pi;
            else:
                bn[i] = (np.sin(wh*n))/(pi*n) - (np.sin(wl*n))/(pi*n)   # Filter impulse response

        bn = bn*np.kaiser(M,5.2)  # apply Kaiser window, alpha= 5.2
        b.append(bn)

        [w,h]= scipy.signal.freqz(bn,1)
        filtered.append(np.convolve(bn, y)) # filter the signal by convolving the signal with filter coefficients

    alpha = filtered[1]
    alpha = np.fft.fft(alpha[M//2:L+M//2])
    theta = filtered[2]
    theta = np.fft.fft(theta[M//2:L+M//2])
    beta = filtered[3]
    beta = np.fft.fft(beta[M//2:L+M//2])

    return alpha.real, theta.real, beta.real



In [ ]:
def eeg_w_beta(signal):
    # calculates eeg_w
    # Chanel et al Emotion assessment from physiological signals for
    # adaptation of game difficulty
    alpha, theta, beta = eeg_bands(signal[4224:])
    nomin = np.sum(beta)
    denom = np.sum(theta + alpha)
    eeg_w = np.log(np.absolute(nomin/denom))
    return eeg_w


In [ ]:
# construct feature vectors
# MIT License

# Copyright (c) 2018 paulmorio

def power_spectrums(signal):
    # calculates the concatenated vector of the power spectra
    # for the alpha, theta, beta bands
    # via the welch method for estimating spectral density
    dk, baseline = scipy.signal.welch(signal[:384], fs=128, scaling = 'spectrum')
    dk, trial_freq = scipy.signal.welch(signal[4224:], fs=128, scaling = 'spectrum')
    c = trial_freq-baseline
    alpha = np.log(np.square(np.mean(c[8:17])))
    theta = np.log(np.square(np.mean(c[16:25])))
    beta = np.log(np.square(np.mean(c[25:61])))
    return [alpha, theta, beta]


In [ ]:
# construct feature vectors
# MIT License

# Copyright (c) 2018 paulmorio

for person in subjects:
    for vid in videos:
        signals_data = (((dummy[person])['data'])[vid])[:32]
        ratings = ((dummy[person]['labels'])[vid])
        y.append(ratings) # append video ratingS to labels

        # Our data vector
        x = []

        # energy log sum ratios eeg_w for all the channels
        eeg_w_list = []
        for signal in signals_data:
            eeg_w_list.append(eeg_w_beta(signal))
        x = x + eeg_w_list

        # powerspectrumstuff
        for signal in signals_data:
            x = x + power_spectrums(signal)

        X.append(x)
    print(person)

In [ ]:
new_feature_vector_file = 'C:\\Users\\q42\\Thesis\\data\\eeg_data_old.dat'
data = save_obj((X, y), new_feature_vector_file)

In [ ]:
# ends part 2 


In [ ]:
# https://gitlab.csl.uni-bremen.de/rlicona
# MIT License
# Copyright (c) 2019 ricardo licona

import pandas as pd
import numpy as np

# classifier
# Machinelearning
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.ensemble import RandomForestClassifier

class classicalClassifiers(object):
    def __init__(self, cv=5, k_pca=5, n_jobs=-1, scoring='f1'):
        self.cv = cv
        self.k_pca = k_pca
        self.n_jobs = n_jobs
        self.scoring = scoring

    def svmClassifier(self, x, y, option):
        pca_result = SelectKBest(f_classif, self.k_pca).fit_transform(x, y)
        pca_result = x
        svmRBFValence = svm.SVC()
        a = cross_val_score(svmRBFValence, pca_result, y, cv=self.cv, scoring=self.scoring, n_jobs=self.n_jobs)
        return np.mean(a)*100, np.std(a)
        
        
class affectiveEmotions(object):

    def __init__(self, valence='v', arousal='a'):
        self.valence = valence
        self.arousal = arousal

    def getPerfomanceValence(self, x, y, classifier):

        if classifier == 'svm':
            performanceValence, stdValence = classicalClassifiers().svmClassifier(x, y, self.valence)
            return performanceValence, stdValence
        else:
            return 0
    def getPerfomanceArousal(self, x, y, classifier):

        if classifier == 'svm':
            performanceArousal, stdArousal = classicalClassifiers().svmClassifier(x, y, self.arousal)
            return performanceArousal, stdArousal
        else:
            return 0


In [ ]:
feature_file_ready = 'C:\\Users\\q42\\Thesis\\data\\eeg_data_old.dat'
X, y = load_obj(feature_file_ready)

In [ ]:
X = np.array(X)
y = np.array(y)


In [ ]:
print("X Shape: \n", X.shape)

X Shape: 
 (1280, 128)


In [ ]:
print("y Shape: \n", y.shape)

y Shape: 
 (1280, 4)


In [ ]:
y

array([[7.71, 7.6 , 6.9 , 7.83],
       [8.1 , 7.31, 7.28, 8.47],
       [8.58, 7.54, 9.  , 7.08],
       ...,
       [3.05, 7.01, 5.1 , 1.1 ],
       [3.99, 7.17, 4.85, 1.  ],
       [7.15, 4.03, 9.  , 1.88]])

In [ ]:
def data_binarizer(ratings, threshold):
    """binarizes the data below and above the threshold"""
    binarized = []
    for rating in ratings:
        if rating <= threshold:
            binarized.append(0)
        else:
            binarized.append(1)
    return binarized

In [ ]:
y_valence = np.array(sp.data_binarizer([el[0] for el in y], 5))
y_arousal = np.array(sp.data_binarizer([el[1] for el in y], 5))

# Describe the data
arousalpd = pd.Categorical(y_arousal)
valencepd = pd.Categorical(y_valence)


In [ ]:
#print("\nY valence shape: \n", y_valence.shape) #(1280,)
performanceValence, stdValence = affectiveEmotions().getPerfomanceValence(X, y_valence, 'svm')
print("\nValence \nPerfomance (F1 metric):  " + "%.2f" % performanceValence)
print("\nSTD: ", stdValence)



Valence 
Perfomance (F1 metric):  63.96

STD:  0.05788996753113706


In [ ]:
#print("\nY arousal shape: \n", y_arousal.shape) #(1280,)
performanceArousal, stdArousal = affectiveEmotions().getPerfomanceArousal(X, y_arousal, 'svm')
print("\nValence \nPerfomance (F1 metric):  " + "%.2f" % performanceArousal)
print("\nSTD: ", stdArousal)



Valence 
Perfomance (F1 metric):  61.68

STD:  0.07825063277133627
